<a href="https://colab.research.google.com/github/manas-1404/Chest-CNN/blob/main/ResNet18_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim


In [2]:
def modify_resnet18():
    # Load a pre-trained ResNet18 model
    model = models.resnet18(pretrained=True)

    # Change the first convolution layer to accept a single channel (grayscale)
    model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

    # Adjust the final fully connected layer to the number of classes in MNIST
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 10)

    return model


In [14]:
transform = transforms.Compose([
    transforms.Resize(224),  # Resize the images to 224x224
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485], std=[0.229])  # Normalizing with ImageNet stats
])

# Load MNIST data
train_dataset = MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = MNIST(root='./data', train=False, transform=transform)

# Create data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=60, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=60, shuffle=False)


In [15]:
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of batches per epoch: {len(train_loader)}")


Number of training samples: 60000
Number of batches per epoch: 1000


In [16]:
def train_model(model, train_loader, epochs=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        total_batches = 0  # Counter to track the number of batches processed
        for images, labels in train_loader:
            # print("-"*75)
            total_batches += 1
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1}, Batch Number {total_batches}, Average Loss: {total_loss/total_batches}")


model = modify_resnet18()

train_model(model, train_loader)


cuda
Epoch 1, Batch Number 1000, Average Loss: 0.06888471301743994
Epoch 2, Batch Number 1000, Average Loss: 0.03332242677410249
Epoch 3, Batch Number 1000, Average Loss: 0.027229200329151354
Epoch 4, Batch Number 1000, Average Loss: 0.02322526342679339
Epoch 5, Batch Number 1000, Average Loss: 0.019878966984259023
Epoch 6, Batch Number 1000, Average Loss: 0.018054614545861113
Epoch 7, Batch Number 1000, Average Loss: 0.013255958643980194
Epoch 8, Batch Number 1000, Average Loss: 0.013592571076880632
Epoch 9, Batch Number 1000, Average Loss: 0.010431886660590863
Epoch 10, Batch Number 1000, Average Loss: 0.009903182845065658


In [18]:
def evaluate_model(model, test_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)
    model = model.to(device)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy of the model on the test images: {100 * correct / total}%')

evaluate_model(model, test_loader)


cuda
Accuracy of the model on the test images: 99.51%
